In [1]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
from dataclasses import dataclass
import duckdb
import json
import io
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

%load_ext autoreload
%autoreload 2

# Styling
#sns.set_style('whitegrid')
#plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
# Oracle Database Integration
# Use this to load data directly from your Oracle database

import oracledb
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

def setup_oracle_session(engine):
    """
    Set up Oracle E-Business Suite session context.
    
    This runs immediately after connecting and:
    - Sets the schema to APPS
    - Configures multi-org context
    
    Required for querying Oracle EBS tables.
    """
    session_setup_sql = text("""
    BEGIN
        EXECUTE IMMEDIATE 'ALTER SESSION SET CURRENT_SCHEMA = APPS';
        apps.XXATD_ORG_ACCESS_UTILS_PKG.SET_multi_ORG_CONTEXT;
    END;
    """)
    
    try:
        with engine.connect() as conn:
            conn.execute(session_setup_sql)
            conn.commit()
            print("✓ Oracle EBS session configured (schema: APPS, org context set)")
    except Exception as e:
        print(f"Warning: Session setup failed: {str(e)}")
        print("Query may fail if schema/context not properly set...")

def load_from_oracle(org_id: int = None, user: str = None, password: str = None, dsn: str = None):
    """
    Load order line data directly from Oracle database using SQLAlchemy.
    
    Parameters:
    -----------
    org_id : int, optional
        Organization ID to filter data (default: from ORACLE_ORG_ID env var)
    user : str, optional
        Oracle database username (default: from ORACLE_USER env var)
    password : str, optional
        Oracle database password (default: from ORACLE_PASSWORD env var)
    dsn : str, optional
        Data Source Name (default: from ORACLE_DSN env var)
        Format: 'hostname:port/service_name'
        Example: 'dbhost.company.com:1521/PROD'
    
    Returns:
    --------
    pandas.DataFrame
        Order line data ready for truck load planning
    
    Note:
    -----
    Create a .env file with your credentials:
        ORACLE_USER=your_username
        ORACLE_PASSWORD=your_password
        ORACLE_DSN=dbhost:1521/PROD
        ORACLE_ORG_ID=123
    """
    
    # Get credentials from parameters or environment variables
    org_id = org_id or int(os.getenv('ORACLE_ORG_ID', '0'))
    user = user or os.getenv('ORACLE_USER')
    password = password or os.getenv('ORACLE_PASSWORD')
    dsn = dsn or os.getenv('ORACLE_DSN')
    
    # Validate required parameters
    if not all([org_id, user, password, dsn]):
        missing = []
        if not org_id: missing.append('org_id/ORACLE_ORG_ID')
        if not user: missing.append('user/ORACLE_USER')
        if not password: missing.append('password/ORACLE_PASSWORD')
        if not dsn: missing.append('dsn/ORACLE_DSN')
        
        raise ValueError(
            f"Missing required parameters: {', '.join(missing)}\n"
            f"Provide via function arguments or create .env file with environment variables."
        )
    
    # Parse DSN
    if ':' in dsn and '/' in dsn:
        host_port, service = dsn.rsplit('/', 1)
        host, port = host_port.rsplit(':', 1)
        port = int(port)
    elif ':' in dsn:
        host, port = dsn.rsplit(':', 1)
        port = int(port)
        service = None
    else:
        host = dsn
        port = 1521
        service = None
    
    # Create SQLAlchemy connection URL
    if service:
        connection_url = URL.create(
            "oracle+oracledb",
            username=user,
            password=password,
            host=host,
            port=port,
            query={"service_name": service}
        )
    else:
        connection_url = URL.create(
            "oracle+oracledb",
            username=user,
            password=password,
            host=host,
            port=port
        )
    
    # Create engine
    engine = create_engine(connection_url)
    
    try:
        print(f"Connecting to Oracle database at {host}...")
        
        # Test connection
        with engine.connect():
            print("✓ Connection successful!")
        
        # Set up Oracle E-Business Suite session context
        setup_oracle_session(engine)
        
        # Your SQL query - NOTE: Priority classification happens in Python after loading
        sql_query = """
        With opendcopenlines
          As (  Select ORDERED_DATE
            , LINE_CATEGORY_CODE
            , ORDERED_ITEM
            , a.inventory_item_id
            , ORIG_SYS_DOCUMENT_REF
            , order_number
            , line_id
            , line_number || '.' || shipment_number line
            , schedule_ship_date
            , ordered_quantity
            , SUM(NVL(RESERVATION_QUANTITY, 0))   reservedqty
            , SHIPPING_METHOD_CODE
            , a.attribute8        iso
            , ATTRIBUTE17
            , LINE_TYPE
            , PRICE_LIST
            , SOLD_TO
            , ship_From
            , SHIP_TO
            , SHIP_TO_ADDRESS1
            , SHIP_TO_ADDRESS5
            , SHIP_SET_ID
            , open_flag
            ,NVL(cancelled_flag, 'N') cancelled_flag
            From oe_order_lines_v a, 
            mtl_reservations mr
           Where   ship_from_org_id = :orgnid
              And open_flag = 'Y'
              And line_id = demand_source_line_id(+)
              And line_type Not In
                ('ATD Bill Only Line'
               , 'ATD Vendor Direct Ship Line'
               , 'ATD STHVendor Direct Ship Line')
              And ordered_date > SYSDATE - 60
           Group By ORDERED_DATE
            , LINE_CATEGORY_CODE
            , ORDERED_ITEM
            , ORIG_SYS_DOCUMENT_REF
            , order_number
            , a.inventory_item_id
            , line_id
            , line_number || '.' || shipment_number
            , schedule_ship_date
            , ordered_quantity
            , SHIPPING_METHOD_CODE
            , ATTRIBUTE17
            , LINE_TYPE
            , PRICE_LIST
            , SOLD_TO
            , ship_From
            , SHIP_TO
            , SHIP_TO_ADDRESS1
            , SHIP_TO_ADDRESS5
            , SHIP_SET_ID
        , open_flag
            ,NVL(cancelled_flag, 'N')    , a.attribute8)
            select * from opendcopenlines
        """
        
        # Execute query
        print(f"Executing query for organization {org_id}...")
        df = pd.read_sql(sql_query, engine, params={'orgnid': org_id})
        
        print(f"✓ Successfully loaded {len(df)} order lines from database")
        
        # Normalize column names to uppercase (Oracle default)
        print("Normalizing column names...")
        df.columns = df.columns.str.upper()
        

        return df
        
    except Exception as e:
        print(f"✗ Error: {str(e)}")
        print(f"\nTroubleshooting:")
        print(f"1. Verify database credentials in .env")
        print(f"2. Check VPN/network connection")
        print(f"3. Verify user has access to APPS schema")
        print(f"4. Confirm org_id {org_id} is correct")
        raise
    
    finally:
        engine.dispose()


# Example usage:
# ============================================================================
# Using .env file (RECOMMENDED)
# ============================================================================

"""
# Create .env file with:
ORACLE_USER=your_username
ORACLE_PASSWORD=your_password
ORACLE_DSN=dbhost.company.com:1521/PROD
ORACLE_ORG_ID=123

# Then call:
df = load_from_oracle()

# Or override specific parameters:
df = load_from_oracle(org_id=456)
"""
df = load_from_oracle(org_id=132)

print("✓ Oracle integration ready!")
print("\nTo use:")
print("1. Ensure .env file is configured with Oracle EBS credentials")
print("2. Uncomment the line below and run this cell:")
print("3. df = load_from_oracle()")
print("\nNote: Session will be automatically configured for APPS schema and multi-org context")

Connecting to Oracle database at ebsprda-scan.atd-us.icd...
✗ Error: (oracledb.exceptions.OperationalError) DPY-6005: cannot connect to database (CONNECTION_ID=rWUWx1JJT5r7ZPu7atR7Lg==).
DPY-4011: the database or network closed the connection
Help: https://python-oracledb.readthedocs.io/en/latest/user_guide/troubleshooting.html#dpy-4011
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Troubleshooting:
1. Verify database credentials in .env
2. Check VPN/network connection
3. Verify user has access to APPS schema
4. Confirm org_id 132 is correct


OperationalError: (oracledb.exceptions.OperationalError) DPY-6005: cannot connect to database (CONNECTION_ID=rWUWx1JJT5r7ZPu7atR7Lg==).
DPY-4011: the database or network closed the connection
Help: https://python-oracledb.readthedocs.io/en/latest/user_guide/troubleshooting.html#dpy-4011
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [52]:
# Register DataFrame directly as a DuckDB table (preserves column types)
duckdb.sql("CREATE OR REPLACE TABLE order_lines AS SELECT * FROM df")

In [62]:
# Assuming your baml_client is inside a dir called app/
import baml_client as client # you can name this "llm" or "baml" or whatever you want
from baml_py import Collector

collector = Collector(name="my-collector")

# Generate SQL from natural language question
response = client.b.GenerateSQL(
    user_question="show me orders by ship method which have reservation and scheduled to ship today, also by order type",
    current_datetime="2025-11-28T14:00:00", baml_options={"collector": collector}
)

# Parse the response
print("=== Generated SQL ===")
print(response.sql)
print("\n=== Explanation ===")
print(response.explanation)
print("\n=== Tables Used ===")
print(response.tables_used)
print("\n=== Filters Applied ===")
print(response.filters_applied)
print("\n=== Metrics Returned ===")
print(response.metrics_returned)

# Execute the generated SQL against DuckDB
print("\n=== Query Results ===")
result_df = duckdb.sql(response.sql).df()
display(result_df)

2025-11-28T10:34:16.558 [BAML INFO] Function GenerateSQL:
    Client: CustomHaiku (claude-3-5-haiku-20241022) - 5970ms. StopReason: end_turn. Tokens(in/out): 1761/388
    ---PROMPT---
    user: Generate a DuckDB SQL query to answer this DC (Distribution Center) dashboard question.
    
    === TABLE SCHEMA ===
    
    TABLE: order_lines
    COLUMNS:
      - ORDERED_DATE: TIMESTAMP (when order was placed)
      - LINE_CATEGORY_CODE: VARCHAR ('ORDER', 'RETURN')
      - ORDERED_ITEM: VARCHAR (SKU/part number, e.g., 'CSN.59000370')
      - INVENTORY_ITEM_ID: INTEGER (internal item ID)
      - ORIG_SYS_DOCUMENT_REF: VARCHAR (external order reference)
      - ORDER_NUMBER: INTEGER (sales order number)
      - LINE_ID: INTEGER (unique line identifier)
      - LINE: VARCHAR (line number within order, e.g., '1.1')
      - SCHEDULE_SHIP_DATE: TIMESTAMP (promised ship date/time)
      - ORDERED_QUANTITY: INTEGER (qty customer ordered)
      - === Generated SQL ===
WITH ShippingMethodBreakdown AS

ParserException: Parser Error: syntax error at end of input

In [63]:
print("=== Generated SQL ===")
print(response.sql)

=== Generated SQL ===
WITH ShippingMethodBreakdown AS (
  SELECT 
    SPLIT_PART(ol.SHIPPING_METHOD_CODE, '_', 2) as carrier,
    ol.LINE_CATEGORY_CODE as order_type,
    COUNT(*) as total_lines,
    SUM(ol.ORDERED_QUANTITY) as total_ordered_qty,
    SUM(ol.RESERVEDQTY) as total_reserved_qty,
    COUNT(DISTINCT ol.ORDER_NUMBER) as unique_orders
  FROM order_lines ol
  WHERE ol.OPEN_FLAG = 'Y'
    AND ol.CANCELLED_FLAG = 'N'
    AND ol.RESERVEDQTY > 0
    AND CAST(ol.SCHEDULE_SHIP_DATE AS DATE) = CURRENT_DATE
  GROUP BY 
    SPLIT_PART(ol.SHIPPING_METHOD_CODE, '_', 2),
    ol.LINE_CATEGORY_CODE
  ORDER BY total_lines DESC


In [64]:
print(collector.last.usage)  # Print usage metrics
print(collector.last.raw_llm_response)  # Print final response as string
# since there may be retries, print the last http response received
print(collector.last.calls[-1].http_response) 


Usage(input_tokens=1761, output_tokens=388, cached_input_tokens=0)
{
  "sql": """
WITH ShippingMethodBreakdown AS (
  SELECT 
    SPLIT_PART(ol.SHIPPING_METHOD_CODE, '_', 2) as carrier,
    ol.LINE_CATEGORY_CODE as order_type,
    COUNT(*) as total_lines,
    SUM(ol.ORDERED_QUANTITY) as total_ordered_qty,
    SUM(ol.RESERVEDQTY) as total_reserved_qty,
    COUNT(DISTINCT ol.ORDER_NUMBER) as unique_orders
  FROM order_lines ol
  WHERE ol.OPEN_FLAG = 'Y'
    AND ol.CANCELLED_FLAG = 'N'
    AND ol.RESERVEDQTY > 0
    AND CAST(ol.SCHEDULE_SHIP_DATE AS DATE) = CURRENT_DATE
  GROUP BY 
    SPLIT_PART(ol.SHIPPING_METHOD_CODE, '_', 2),
    ol.LINE_CATEGORY_CODE
  ORDER BY total_lines DESC
""",
  "explanation": "Analyzes orders scheduled to ship today with existing reservations, breaking down by shipping carrier and order type",
  "tables_used": ["order_lines"],
  "filters_applied": [
    "Active orders only",
    "Not cancelled",
    "Has reservations",
    "Scheduled to ship today"
  ],
  "met

In [60]:
# Assuming your baml_client is inside a dir called app/
import baml_client as client # you can name this "llm" or "baml" or whatever you want
from baml_py import Collector

collector = Collector(name="my-collector1")

# Generate SQL from natural language question
response = client.b.GenerateIssueScanSQL(
    current_datetime="2025-11-28T14:00:00", baml_options={"collector": collector}
)

# Parse the response
print("=== Generated SQL ===")
print(response.sql)
print("\n=== Explanation ===")
print(response.explanation)
print("\n=== Tables Used ===")
print(response.tables_used)
print("\n=== Filters Applied ===")
print(response.filters_applied)
print("\n=== Metrics Returned ===")
print(response.metrics_returned)

# Execute the generated SQL against DuckDB
print("\n=== Query Results ===")
result_df = duckdb.sql(response.sql).df()
display(result_df)

2025-11-28T10:23:47.388 [BAML INFO] Function GenerateIssueScanSQL:
    Client: CustomHaiku (claude-3-5-haiku-20241022) - 15560ms. StopReason: end_turn. Tokens(in/out): 649/1351
    ---PROMPT---
    user: Generate a comprehensive SQL query to detect all current issues in the DC.
    
    CURRENT DATETIME: 2025-11-28T14:00:00
    
    TABLE: order_lines (same schema as above)
    
    ISSUES TO DETECT:
    
    1. CRITICAL - Past due backorders (overdue + not reserved)
       - SCHEDULE_SHIP_DATE < CURRENT_TIMESTAMP
       - RESERVEDQTY < ORDERED_QUANTITY
       - Label: 'Past Due Backorder'
    
    2. CRITICAL - Past due with available inventory (should have shipped)
       - SCHEDULE_SHIP_DATE < CURRENT_TIMESTAMP
       - QTYINHUB > 0 OR QTYINLOCALPLUS > 0
       - Label: 'Past Due - Stock Available'
    
    3. WARNING - Due today, not reserved
       - CAST(SCHEDULE_SHIP_DATE AS DATE) = CURRENT_DATE
       - RESERVEDQTY < ORDERED_=== Generated SQL ===
WITH current_issues AS (
  SELE

BinderException: Binder Error: Cannot compare values of type TIMESTAMP_NS and type TIMESTAMP WITH TIME ZONE - an explicit cast is required